# 丢弃

In [1]:
import pandas as pd

In [2]:
df_cell_blind = pd.read_csv('../data_KEGG186/drug_sensitivity_lung_CellBlind_test2&prediction2.csv')
print(df_cell_blind.shape)
df_cell_blind.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data_KEGG186/drug_sensitivity_lung_CellBlind_test2&prediction2.csv'

In [ ]:
# 筛选出OncotreeCode为LUAD、LUSC、SCLC、NSCLC、LCLC的数据
# df_LUNG = df_cell_blind[df_cell_blind['OncotreeCode'].isin(['LUAD', 'LUSC', 'SCLC', 'NSCLC', 'LCLC'])]
df_LUNG = df_cell_blind[df_cell_blind['OncotreeCode'].isin(['LUAD', 'LUSC', 'SCLC', 'LCLC'])]
print(df_LUNG.shape)
df_LUNG.head()

In [ ]:
# 剔除cell_line为NCIH1355、EKVX的数据    'NCIH1355', 'EKVX'
# df_LUNG = df_LUNG[~df_LUNG['cell_line'].isin(['NCIH441'])]

In [ ]:

# df_LUNG根据cell_line分组计数，结果显示每个cell_line有多少个drug，并且保留OncotreeCode
df_LUNG_count = df_LUNG.groupby(['cell_line', 'OncotreeCode']).count().reset_index()[['cell_line', 'OncotreeCode', 'drug']]
df_LUNG_count.head(20)


In [ ]:
# 将df_LUNG根据cell_line分组，再取每组drug列的交集
df_LUNG_drug = df_LUNG.groupby('cell_line')['drug'].apply(set).reset_index()
df_LUNG_drug.head(20)

In [ ]:
# 将df_LUNG_drug中的drug取交集
drugs = set.intersection(*df_LUNG_drug['drug'])
print(len(drugs))
print(drugs)

In [ ]:
# 取df_LUNG中drug中在drugs中的数据
df_LUNG_Filtered = df_LUNG[df_LUNG['drug'].isin(drugs)][['drug', 'cell_line', 'prediction', 'OncotreeCode','IC50']]
print(df_LUNG_Filtered.shape)
df_LUNG_Filtered.head()

In [ ]:
df_drug_mean_std = pd.read_csv('../data/gdsc_drug_mean_std.csv')
df_drug_mean_std.head()

In [ ]:
# 根据df_drug_mean_std中的"Drug_Name"列、"IC50_mean"列和"IC50_std"列计算出df_LUAD的Z-Score
df_LUNG_Filtered_ZScore = pd.merge(df_LUNG_Filtered, df_drug_mean_std, left_on='drug', right_on='Drug_Name')
df_LUNG_Filtered_ZScore['Z-Score'] = (df_LUNG_Filtered_ZScore['prediction'] - df_LUNG_Filtered_ZScore['IC50_mean']) / df_LUNG_Filtered_ZScore['IC50_std']
df_LUNG_Filtered_ZScore.head()

In [ ]:
#####测试代码########
# 统计每种药物的Z-Score的均值
df_LUNG_Filtered_ZScore_mean = df_LUNG_Filtered_ZScore.groupby('drug')['Z-Score'].mean().reset_index()
df_LUNG_Filtered_ZScore_mean.head(200)



In [ ]:
# 剔除cell_line为NCIH1355、EKVX的数据
# df_LUNG_Filtered_ZScore = df_LUNG_Filtered_ZScore[~df_LUNG_Filtered_ZScore['cell_line'].isin(['NCIH1355', 'EKVX'])]
# df_LUNG_Filtered_ZScore = df_LUNG_Filtered_ZScore[~df_LUNG_Filtered_ZScore['cell_line'].isin(['NCIH2228', 'HCC827'])]

In [ ]:
df_LUNG_Filtered_ZScore['cell_line'] = df_LUNG_Filtered_ZScore['OncotreeCode'] + '.' + df_LUNG_Filtered_ZScore['cell_line']

In [ ]:
# 使用pivot函数，将df_LUNG_Filtered_ZScore的drug列作为行索引，cell_line列作为列索引，prediction列作为值
df_LUNG_Filtered_ZScore_pivot = df_LUNG_Filtered_ZScore.pivot(index='drug', columns='cell_line', values='Z-Score')
df_LUNG_Filtered_ZScore_pivot.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
# iris = sns.load_dataset("iris")
# species = iris.pop("species")
# heatmap_data = pd.read_csv('heatmap_data.csv')
cell_line_names = df_LUNG_Filtered_ZScore_pivot.columns
heatmap_data = df_LUNG_Filtered_ZScore_pivot.iloc[:,0:]
#设置图片大小
g= sns.clustermap(heatmap_data,row_cluster=False,col_cluster=True, metric='euclidean',
                  fmt="d",cmap='Spectral_r',figsize=(5,10),
                  dendrogram_ratio=(.1, .2),
                    cbar_pos=(0, .2, .03, .4)
                  , yticklabels=False)
ax = g.ax_heatmap
label_y = ax.get_yticklabels()
plt.setp(label_y, rotation=360, horizontalalignment='left')
label_x = ax.get_xticklabels()
plt.setp(label_x, rotation=50, horizontalalignment='right')
#设置图片名称，分辨率，并保存
# plt.savefig('cluster.tif',dpi = 300)
plt.show()